In [1]:
%load_ext autoreload

In [35]:
%autoreload 2
import numpy as np
import pylab as plt

import odynn
from odynn.circuit import Circuit
from odynn.models import LeakyIntegrate, ChemSyn, GapJunction


from ipywidgets import interact

p = LeakyIntegrate.default_params
dt = 20.

@interact(g_gap = 0., g12=0.5, g21=0.5, mdp12=-40, mdp21=-40, scale12=0.01, scale21=0.01, E12=-80, E21=-80)
def dual(g_gap, g12, g21, mdp12, mdp21, scale12, scale21, E12, E21):
    connections = [(0,1) , (1,0)]
    t = np.arange(0.0, 5000., dt)
    i0 = np.array([10. *((t>i)&(t<i+500)) for i in range(0,5000,1000)]).sum(0)
    i1 = np.array([10. *((t>i-500)&(t<i)) for i in range(1000,5000,1000)]).sum(0)
    i_injs = np.stack((i0, i1), axis=1)[:,None,:,None]
    
    neur = {'C_m': np.array([[0.2], [0.2]]), 
            'g_L': np.array([[0.5], [0.5]]), 
            'E_L': np.array([[-50.], [-50.]])}
    syns = {'G': np.array([[g12], [g21]]), 
            'mdp': np.array([[mdp12], [mdp21]]), 
            'scale': np.array([[scale12], [scale21]]), 
            'E': np.array([[E12], [E21]])}
    gap = {'G_gap' : np.array([[g_gap]]),
          'factor' : [[1]]}
    n = LeakyIntegrate(init_p=neur)
    s = ChemSyn([c[0] for c in connections], [c[1] for c in connections], init_p = syns)
    g = GapJunction([0], [1], init_p=gap)
    circuit = Circuit(n, s, g)
    states = circuit.calculate(i_injs)
    ax = plt.subplot(211)
    plt.title('Voltage (mV)')
    plt.plot(states[:,0,0,:,0])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax = plt.subplot(212)
    plt.plot(i_injs[:,0,:,0])
    plt.xlabel('time (s)')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.show()
    

interactive(children=(FloatSlider(value=0.0, description='g_gap', max=1.0), FloatSlider(value=0.5, description…

In [3]:
import torch

torch.clip = torch.clamp

a = torch.rand(5)
print(a)
print(torch.clip(a, 0.4, 0.6))

tensor([0.8861, 0.6817, 0.2635, 0.0582, 0.2134])
tensor([0.6000, 0.6000, 0.4000, 0.4000, 0.4000])
